In [7]:
import mysql.connector
from mysql.connector import Error
from db_connect import DB_CONFIG
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib


/tmp/ipykernel_33892/621697402.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
def connect_to_database():
    global connection, cursor
    try:
        connection = mysql.connector.connect(
            host=DB_CONFIG['host'],
            database=DB_CONFIG['database'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password']
        )

        if connection.is_connected():
            print("Successfully connected to the database")
            cursor = connection.cursor()

            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            print("Tables in the database:")
            for table in tables:
                print(table[0])

            return connection

    except Error as e:
        print(f"Error: {e}")

def search_query(query) :
    cursor.execute(query)
    rows = cursor.fetchall()
    data_list = []
    for row in rows :
        data_list.append([*row])
    return data_list

In [9]:
connect_to_database()

Successfully connected to the database
Tables in the database:
auction_price_data
radioactive_test
radioactive_test1
radioactivity_pollution
test


In [9]:
query = """
SELECT * FROM auction_price_data ORDER BY date LIMIT 5;
"""
df = pd.DataFrame(search_query(query))
df

,0,1,2,3,4,5,6,7,8
0,(선)깐홍합,여수,소,box,426.0,13500.0,11000.0,12400.0,2011-01-03
1,(선)가자미,보령,24미,c/s(상자),10.0,12000.0,12000.0,12000.0,2011-01-03
2,(활)농어,격포,중,kg,98.9,3700.0,3700.0,3700.0,2011-01-03
3,(선)깐굴,통영,중,box,10.0,40000.0,35000.0,37500.0,2011-01-03
4,(냉)갈치,중국,250/350,CT/(BT),10.0,40000.0,40000.0,40000.0,2011-01-03


In [63]:
query = """
SELECT
    species,
    DATE_FORMAT(date, '%Y-%m') AS month,
    SUM(quantity * average) AS total_amount,
    SUM(quantity) AS total_quantity,
    SUM(quantity * average) / SUM(quantity) AS monthly_weighted_average
FROM
    auction_price_data
WHERE
    species LIKE "(활)암꽃게" AND
    date BETWEEN '2015-01-01' AND '2023-12-31'
GROUP BY
    species,
    month
ORDER BY
    month;
"""

In [64]:
df = pd.DataFrame(search_query(query))
df.columns = ['species', 'month', 'total', 'quantity', 'average']
df.head(10)

,species,month,total,quantity,average
0,(활)암꽃게,2015-01,1.439637e+08,5674.000000,25372.523793
1,(활)암꽃게,2015-02,1.241000e+06,57.000000,21771.929825
2,(활)암꽃게,2015-03,2.145008e+08,8891.000000,24125.610168
3,(활)암꽃게,2015-04,2.032529e+09,93644.000000,21704.852420
4,(활)암꽃게,2015-05,2.611885e+09,105593.500000,24735.283895
5,(활)암꽃게,2015-06,9.140485e+08,44158.900024,20699.077874
6,(활)암꽃게,2015-08,1.758450e+07,2010.000000,8748.507463
7,(활)암꽃게,2015-09,3.894965e+08,36696.000000,10614.140506
8,(활)암꽃게,2015-10,2.060290e+09,170779.000000,12064.070231
9,(활)암꽃게,2015-11,3.996424e+08,17162.000000,23286.470108


In [ ]:
# df.columns = ['species', 'origin', 'size', 'packaging', 'quantity', 'highest', 'lowest', 'average', 'date']

In [6]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 4))
plt.plot(average_prices['month_year'], average_prices['average'], marker='o')
plt.title('물품들의 날짜별 평균 가격 변화')
plt.xlabel('날짜')
plt.ylabel('평균가')
plt.ylim(0,50000)
plt.grid(True)
xticks = average_prices['month_year'][::2]
plt.xticks(xticks, rotation=90)
plt.tight_layout()
plt.show()

NameError: name 'average_prices' is not defined

<Figure size 1800x400 with 0 Axes>

In [61]:
if connection.is_connected():
    connection.close()
    print("MySQL connection is closed")

MySQL connection is closed


In [ ]:
all_months = pd.date_range(start=average_prices['month_year'].min(), end=average_prices['month_year'].max(), freq='MS')
average_prices_reindexed = average_prices.set_index('month_year').reindex(all_months).reset_index()

# 누락된 월을 0으로 채움 (또는 다른 값으로 채움)
average_prices_reindexed['average'].fillna(0, inplace=True)

plt.figure(figsize=(18, 4))
plt.plot(average_prices_reindexed['month_year'], average_prices_reindexed['average'], marker='o')
plt.title('물품들의 날짜별 평균 가격 변화')
plt.xlabel('날짜')
plt.ylabel('평균가')
plt.ylim(0, 50000)
plt.grid(True)
xticks = average_prices_reindexed['month_year'][::2]
plt.xticks(xticks, rotation=90)
plt.tight_layout()
plt.show()

In [1]:
def generate_query(table, colors=None, size=None, start_date=None, end_date=None, brands=None):
    base_query = f"SELECT * FROM {table} WHERE"
    conditions = []

    if colors:
        color_condition = " OR ".join([f"COLOR = '{color}'" for color in colors])
        conditions.append(f"({color_condition})")

    if size:
        conditions.append(f"SIZE = '{size}'")

    if start_date and end_date:
        conditions.append(f"Release_date BETWEEN '{start_date}' AND '{end_date}'")

    if brands:
        brand_condition = " OR ".join([f"Brand = '{brand}'" for brand in brands])
        conditions.append(f"({brand_condition})")

    query = f"{base_query} {' AND '.join(conditions)} GROUP BY NAME ORDER BY PRICE ASC LIMIT 100"

    return query

# 예제 실행
table = "T_shirt"
colors = ["WHITE", "RED"]
size = "L"
start_date = "2020-01-01"
end_date = "2023-12-31"
brands = ["Puma"]

sql_query = generate_query(table, colors, size, start_date, end_date, brands)
print(sql_query)

SELECT * FROM T_shirt WHERE (COLOR = 'WHITE' OR COLOR = 'RED') AND SIZE = 'L' AND Release_date BETWEEN '2020-01-01' AND '2023-12-31' AND (Brand = 'Puma') GROUP BY NAME ORDER BY PRICE ASC LIMIT 100


In [ ]:
"""
SELECT *
FROM T_shirt
WHERE (COLOR = 'WHITE' OR COLOR = 'RED')
    AND SIZE = 'L'
    AND Release_date BETWEEN '2020-01-01' AND '2023-12-31'
    AND (Brand = 'Puma')
GROUP BY NAME
ORDER BY PRICE ASC LIMIT 100
"""

"""
SELECT
    species,
    DATE_FORMAT(date, '%Y-%m') AS month,
    SUM(quantity * average) AS total_amount,
    SUM(quantity) AS total_quantity,
    SUM(quantity * average) / SUM(quantity) AS monthly_weighted_average
FROM auction_price_data
WHERE species LIKE "(활)암꽃게" 
    AND date BETWEEN '2015-01-01' AND '2023-12-31'
GROUP BY
    species,
    month
ORDER BY month;
"""

In [20]:

# 예제 실행
table = "auction_price_data"
species = ["넙치", "암꽃게"]
size = "대"
start_date = "2020-01-01"
end_date = "2023-12-31"
origins = ["태안"]
keywords = ["활"]

def generate_query(table, species=None, size=None, start_date=None, end_date=None, origins=None, keywords=None):
    base_query = f"SELECT * FROM {table} WHERE"
    conditions = []

    if keywords: #(활), (냉), (선)
        keywords_condition = " OR ".join([f"species LIKE '({keyword})%'" for keyword in keywords])
        conditions.append(f"({keywords_condition})")

    if species:
        color_condition = " OR ".join([f"species LIKE '%{specie}%'" for specie in species])
        conditions.append(f"({color_condition})")

    if size:
        conditions.append(f"size = '{size}'")

    if start_date and end_date:
        if start_date > end_date:
            raise ValueError("Start date cannot be after end date")
        conditions.append(f"date BETWEEN '{start_date}' AND '{end_date}'")

    if origins: #태안, 목포, 제주 등
        origin_condition = " OR ".join([f"origin = '{origin}'" for origin in origins])
        conditions.append(f"({origin_condition})")


    query = f"{base_query} {' AND '.join(conditions)} ORDER BY date DESC LIMIT 100"

    return query

sql_query = generate_query(table, species, size, start_date, end_date, origins, keywords)
print(sql_query)

SELECT * FROM auction_price_data WHERE (species LIKE '(활)%') AND (species LIKE '%넙치%' OR species LIKE '%암꽃게%') AND size = '대' AND date BETWEEN '2020-01-01' AND '2023-12-31' AND (origin = '태안') ORDER BY date DESC LIMIT 100


In [22]:
query = """
SELECT * FROM auction_price_data WHERE (species LIKE '(활)%') AND (species LIKE '%넙치%' OR species LIKE '%암꽃게%') AND size = '대' AND date BETWEEN '2020-01-01' AND '2023-12-31' AND (origin = '태안') ORDER BY date DESC LIMIT 100
"""
df = pd.DataFrame(search_query(query))
df

,0,1,2,3,4,5,6,7,8
0,(활)암꽃게,태안,대,kg,31.0,36000.0,35000.0,35200.0,2023-12-30
1,(활)암꽃게,태안,대,kg,54.0,32000.0,29000.0,30000.0,2023-12-29
2,(활)암꽃게,태안,대,kg,47.0,33500.0,31500.0,32600.0,2023-12-28
3,(활)암꽃게,태안,대,kg,104.0,33000.0,25000.0,31000.0,2023-12-27
4,(활)암꽃게,태안,대,kg,290.0,32000.0,23000.0,26400.0,2023-12-26
...,...,...,...,...,...,...,...,...,...
95,(활)암꽃게,태안,대,kg,117.0,11000.0,8500.0,9600.0,2023-08-31
96,(활)암꽃게,태안,대,kg,27.0,11000.0,11000.0,11000.0,2023-08-29
97,(활)암꽃게,태안,대,kg,203.0,8500.0,4500.0,5600.0,2023-08-28
98,(활)암꽃게,태안,대,kg,215.0,25000.0,20000.0,21000.0,2023-08-26
